 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="#Select-SNPs-and-apply-recommended-hard-filters" data-toc-modified-id="Select-SNPs-and-apply-recommended-hard-filters-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Select SNPs and apply recommended hard filters</a></span></li><li><span><a href="#Normalize-vcf" data-toc-modified-id="Normalize-vcf-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Normalize vcf</a></span></li><li><span><a href="#Run-ANNOVAR" data-toc-modified-id="Run-ANNOVAR-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Run ANNOVAR</a></span></li><li><span><a href="#Select-gnomAD-MAF-<-0.01" data-toc-modified-id="Select-gnomAD-MAF-<-0.01-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Select gnomAD MAF &lt; 0.01</a></span></li><li><span><a href="#Select-AC->-1" data-toc-modified-id="Select-AC->-1-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Select AC &gt; 1</a></span></li><li><span><a href="#Run-binomial-test" data-toc-modified-id="Run-binomial-test-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Run binomial test</a></span></li><li><span><a href="#Filter-based-on-glioma-data" data-toc-modified-id="Filter-based-on-glioma-data-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Filter based on glioma data</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import pylab as py
import scipy.stats as stats
from scipy.stats import binom_test
from statsmodels.sandbox.stats.multicomp import multipletests

In [2]:
cromwell_path = '/mnt/chr10/Data/magda/Powroty/panel/cromwell-executions/'

# Select SNPs and apply recommended hard filters

In [12]:
!/home/magda/gatk-4.0.6.0/gatk CountVariants -V "$cromwell_path"JointGenotyping/604f468b-2e1f-42d4-8b02-c4b025130bba/call-FinalGatherVcf/execution/Lib1-6.without113.norm.vcf.gz

Using GATK jar /home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar CountVariants -V /mnt/chr10/Data/magda/Powroty/panel/cromwell-executions/JointGenotyping/604f468b-2e1f-42d4-8b02-c4b025130bba/call-FinalGatherVcf/execution/Lib1-6.without113.norm.vcf.gz
12:00:57.002 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
12:00:57.181 INFO  CountVariants - ------------------------------------------------------------
12:00:57.182 INFO  CountVariants - The Genome Analysis Toolkit (GATK) v4.0.6.0
12:00:57.182 INFO  CountVariants - For support and documentation go to https://software.broadinstitute.org/gatk/
12:00:57.182 INFO  CountVariants - E

In [15]:
select_log = !/home/magda/gatk-4.0.6.0/gatk SelectVariants \
            -V "$cromwell_path"JointGenotyping/604f468b-2e1f-42d4-8b02-c4b025130bba/call-FinalGatherVcf/execution/Lib1-6.without113.norm.vcf.gz \
            --select-type-to-include SNP \
            -O out_files/Lib1-6.no113.SNPs.vcf.gz

In [16]:
!/home/magda/gatk-4.0.6.0/gatk CountVariants -V out_files/Lib1-6.no113.SNPs.vcf.gz

Using GATK jar /home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar CountVariants -V out_files/Lib1-6.no113.SNPs.vcf.gz
12:05:53.502 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
12:05:53.675 INFO  CountVariants - ------------------------------------------------------------
12:05:53.675 INFO  CountVariants - The Genome Analysis Toolkit (GATK) v4.0.6.0
12:05:53.675 INFO  CountVariants - For support and documentation go to https://software.broadinstitute.org/gatk/
12:05:53.676 INFO  CountVariants - Executing as magda@lamina on Linux v4.4.0-31-generic amd64
12:05:53.676 INFO  CountVariants - Java runtime: Java HotSpot(TM) 64-Bit Ser

In [17]:
#https://gatkforums.broadinstitute.org/gatk/discussion/2806/howto-apply-hard-filters-to-a-call-set
filter_log = !/home/magda/gatk-4.0.6.0/gatk VariantFiltration \
    -R /mnt/chr11/Data/magda/gatk-ref/Homo_sapiens_assembly38.fasta \
    -V out_files/Lib1-6.no113.SNPs.vcf.gz \
    -O out_files/Lib1-6.no113.SNPs.hard_filtered.vcf.gz \
    --filter-expression "QD < 2.0 || FS > 60.0 || MQ < 40.0 || MQRankSum < -12.5 || ReadPosRankSum < -8.0" \
    --filter-name "recommended_snp_filter" 

In [20]:
select_log = !/home/magda/gatk-4.0.6.0/gatk SelectVariants \
            -V out_files/Lib1-6.no113.SNPs.hard_filtered.vcf.gz \
            --exclude-filtered \
            -O out_files/Lib1-6.no113.SNPs.hard_filtered.selected.vcf.gz

In [21]:
!/home/magda/gatk-4.0.6.0/gatk CountVariants -V out_files/Lib1-6.no113.SNPs.hard_filtered.selected.vcf.gz

Using GATK jar /home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar CountVariants -V out_files/Lib1-6.no113.SNPs.hard_filtered.selected.vcf.gz
12:11:36.296 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
12:11:36.501 INFO  CountVariants - ------------------------------------------------------------
12:11:36.502 INFO  CountVariants - The Genome Analysis Toolkit (GATK) v4.0.6.0
12:11:36.502 INFO  CountVariants - For support and documentation go to https://software.broadinstitute.org/gatk/
12:11:36.502 INFO  CountVariants - Executing as magda@lamina on Linux v4.4.0-31-generic amd64
12:11:36.503 INFO  CountVariants - Java runtime: Java

# Normalize vcf

In [22]:
!bcftools norm -m-both \
-f /mnt/chr11/Data/magda/gatk-ref/Homo_sapiens_assembly38.fasta \
-o out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.vcf \
out_files/Lib1-6.no113.SNPs.hard_filtered.selected.vcf.gz

Lines total/modified/skipped:	185793/0/0


# Run ANNOVAR

In [23]:
!perl /home/magda/annovar/table_annovar.pl  \
out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.vcf  \
/home/magda/annovar/humandb/ -buildver hg38 -remove  \
-protocol gnomad_genome,avsnp150  \
-operation f,f -nastring .  \
-vcfinput  \
-out out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized


NOTICE: Running with system command <convert2annovar.pl -includeinfo -allsample -withfreq -format vcf4 out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.vcf > out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.avinput>
NOTICE: Finished reading 190544 lines from VCF file
NOTICE: A total of 187122 locus in VCF file passed QC threshold, representing 186188 SNPs (120188 transitions and 66000 transversions) and 0 indels/substitutions
NOTICE: Finished writing allele frequencies based on 4282324 SNP genotypes (2764324 transitions and 1518000 transversions) and 0 indels/substitutions for 23 samples

NOTICE: Running with system command </home/magda/annovar/table_annovar.pl out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.avinput /home/magda/annovar/humandb/ -buildver hg38 -remove -protocol gnomad_genome,avsnp150 -operation f,f -nastring . -outfile out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized -otherinfo>
--------------------------------------

# Select gnomAD MAF < 0.01

In [25]:
annotations = ['gnomAD_genome_ALL',
             'gnomAD_genome_AFR',
             'gnomAD_genome_AMR',
             'gnomAD_genome_ASJ',
             'gnomAD_genome_EAS',
             'gnomAD_genome_FIN',
             'gnomAD_genome_NFE',
             'gnomAD_genome_OTH']

with open('out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.nomissing.vcf', 'w') as o:
    for line in open('out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.vcf').readlines():    
        for el in annotations:
            if el + '=.' in line:
                line = line.replace(el + '=.', el + '=100.0')
        o.write(line)

In [26]:
  select_log = !/home/magda/gatk-4.0.6.0/gatk SelectVariants \
            -V out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.nomissing.vcf \
            -select 'gnomAD_genome_ALL < 0.01' \
            -select 'gnomAD_genome_AFR < 0.01' \
            -select 'gnomAD_genome_AMR < 0.01' \
            -select 'gnomAD_genome_ASJ < 0.01' \
            -select 'gnomAD_genome_EAS < 0.01' \
            -select 'gnomAD_genome_FIN < 0.01' \
            -select 'gnomAD_genome_NFE < 0.01' \
            -select 'gnomAD_genome_OTH < 0.01' \
            -O out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.nomissing.gnomad_below_0.01.vcf

In [27]:
!/home/magda/gatk-4.0.6.0/gatk CountVariants -V out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.nomissing.gnomad_below_0.01.vcf

Using GATK jar /home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar CountVariants -V out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.nomissing.gnomad_below_0.01.vcf
12:19:40.039 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
12:19:40.210 INFO  CountVariants - ------------------------------------------------------------
12:19:40.210 INFO  CountVariants - The Genome Analysis Toolkit (GATK) v4.0.6.0
12:19:40.210 INFO  CountVariants - For support and documentation go to https://software.broadinstitute.org/gatk/
12:19:40.211 INFO  CountVariants - Executing as magda@lamina on Linux v4.4.0-31-generic amd64
12

# Select AC > 1

In [28]:
select_log = !/home/magda/gatk-4.0.6.0/gatk SelectVariants \
            -V out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.nomissing.gnomad_below_0.01.vcf \
            -select 'AC > 1' \
            -O out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.nomissing.gnomad_below_0.01.ac_above_1.vcf

In [29]:
!/home/magda/gatk-4.0.6.0/gatk CountVariants -V out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.nomissing.gnomad_below_0.01.ac_above_1.vcf

Using GATK jar /home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar CountVariants -V out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.nomissing.gnomad_below_0.01.ac_above_1.vcf
12:21:23.397 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/magda/gatk-4.0.6.0/gatk-package-4.0.6.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
12:21:23.596 INFO  CountVariants - ------------------------------------------------------------
12:21:23.596 INFO  CountVariants - The Genome Analysis Toolkit (GATK) v4.0.6.0
12:21:23.596 INFO  CountVariants - For support and documentation go to https://software.broadinstitute.org/gatk/
12:21:23.597 INFO  CountVariants - Executing as magda@lamina on Linux v4.4.0-31-gener

# Run binomial test

In [30]:
totable_log = !/home/magda/gatk-4.0.6.0/gatk VariantsToTable \
    -V out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.nomissing.gnomad_below_0.01.ac_above_1.vcf \
    -F CHROM -F POS -F ID -F REF -F ALT -F AC -F AF -F AN -F avsnp150 \
    -F gnomAD_genome_ALL -F gnomAD_genome_NFE \
    -O out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.nomissing.gnomad_below_0.01.ac_above_1.csv

In [31]:
df = pd.read_csv('out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.nomissing.gnomad_below_0.01.ac_above_1.csv', sep = '\t')

In [32]:
df[:10]

,CHROM,POS,ID,REF,ALT,AC,AF,AN,avsnp150,gnomAD_genome_ALL,gnomAD_genome_NFE
0,chr1,939561,.,G,A,2,0.043,46,rs888928964,0.000052,0.000000
1,chr1,1034902,.,A,T,3,0.065,46,rs751320169,0.001300,0.001700
2,chr1,1779742,rs143227614,G,A,2,0.043,46,rs143227614,0.002400,0.003700
3,chr1,1808285,.,G,A,2,0.091,22,.,0.000065,0.000067
4,chr1,1848381,.,G,A,2,0.045,44,rs914213962,0.000200,0.000200
5,chr1,1859689,.,T,G,2,0.045,44,.,0.003400,0.002400
6,chr1,2072334,.,C,G,4,0.087,46,rs565468909,0.004400,0.007800
7,chr1,2227533,.,C,T,2,0.043,46,.,0.000100,0.000300
8,chr1,2643029,.,T,C,2,0.043,46,rs570074090,0.003900,0.007000
9,chr1,3425762,rs200643126,G,C,2,0.043,46,rs200643126,0.003200,0.004700


In [33]:
print "How many variants have missing gnomAD_genome_NFE info?", len(df[df['gnomAD_genome_NFE']==100])

How many variants have missing gnomAD_genome_NFE info? 0


In [34]:
def binom(row):
        x = row['AC']
        n = row['AN']
        try:
            p = float(row['gnomAD_genome_NFE'])
        except:
            print "error at:", row['CHROM'], row['POS']
        

        return binom_test(x,n,p, alternative = 'greater')       
            
            
df['pval']=df.apply(binom, axis=1)

In [35]:
df.head()

,CHROM,POS,ID,REF,ALT,AC,AF,AN,avsnp150,gnomAD_genome_ALL,gnomAD_genome_NFE,pval
0,chr1,939561,.,G,A,2,0.043,46,rs888928964,0.000052,0.000000,0.000000
1,chr1,1034902,.,A,T,3,0.065,46,rs751320169,0.001300,0.001700,0.000071
2,chr1,1779742,rs143227614,G,A,2,0.043,46,rs143227614,0.002400,0.003700,0.012719
3,chr1,1808285,.,G,A,2,0.091,22,.,0.000065,0.000067,0.000001
4,chr1,1848381,.,G,A,2,0.045,44,rs914213962,0.000200,0.000200,0.000038


In [41]:
multipletests_correction = multipletests(df['pval'], alpha=0.01, 
              method='fdr_bh', is_sorted=False, returnsorted=False)
df['B-H_reject_H0'] = multipletests_correction[0]
df['corrected_p_value'] = multipletests_correction[1]

In [42]:
df.sort_values(by = "pval", ascending=False).head()

,CHROM,POS,ID,REF,ALT,AC,AF,AN,avsnp150,gnomAD_genome_ALL,gnomAD_genome_NFE,pval,B-H_reject_H0,corrected_p_value
1736,chr14,64611478,rs138501516,G,A,2,0.043,46,rs138501516,0.0064,0.0099,0.076205,False,0.076205
1660,chr13,48976478,.,G,A,2,0.043,46,rs568959452,0.0062,0.0099,0.076205,False,0.076205
1505,chr11,117548865,rs138056897,C,T,2,0.043,46,rs138056897,0.0061,0.0099,0.076205,False,0.076205
57,chr1,30909861,rs79374019,T,C,2,0.043,46,rs79374019,0.0068,0.0099,0.076205,False,0.076205
2482,chrX,71182704,.,T,C,2,0.043,46,rs758052869,0.0054,0.0099,0.076205,False,0.076205


In [43]:
len(df[df['B-H_reject_H0']])

1575

In [44]:
df[df['B-H_reject_H0']]['AC'].value_counts()

2     1011
3      297
4      119
5       47
6       32
7       17
8       13
10       9
9        6
12       5
15       3
13       2
11       2
14       2
21       2
24       2
38       1
16       1
17       1
18       1
19       1
43       1
Name: AC, dtype: int64

In [46]:
df.to_csv('out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.nomissing.gnomad_below_0.01.ac_above_1.FDR0.01.csv', sep='\t', index=False)

# Filter based on glioma data

In [47]:
variants_for_filtering = pd.read_csv('../2019-05-07-Prepare_Symfonia_vcf_for_filtering/out_files/symfonia_variants_common.csv', sep='\t')
variants_for_filtering.head()

,CHROM,POS,REF,ALT,Num variant
0,chr1,19217,G,C,5
1,chr1,19355,G,A,5
2,chr1,20129,C,T,13
3,chr1,20144,G,A,7
4,chr1,20166,A,G,8


In [48]:
df_common_merged = pd.merge(df, variants_for_filtering, how = 'outer', on = ['CHROM', 'POS', 'REF', 'ALT'], indicator = True)
df_common_merged.head()

,CHROM,POS,ID,REF,ALT,AC,AF,AN,avsnp150,gnomAD_genome_ALL,gnomAD_genome_NFE,pval,B-H_reject_H0,corrected_p_value,Num variant,_merge
0,chr1,939561,.,G,A,2.0,0.043,46.0,rs888928964,0.000052,0.000000,0.000000,True,0.000000,NaN,left_only
1,chr1,1034902,.,A,T,3.0,0.065,46.0,rs751320169,0.001300,0.001700,0.000071,True,0.000235,NaN,left_only
2,chr1,1779742,rs143227614,G,A,2.0,0.043,46.0,rs143227614,0.002400,0.003700,0.012719,False,0.017786,NaN,left_only
3,chr1,1808285,.,G,A,2.0,0.091,22.0,.,0.000065,0.000067,0.000001,True,0.000005,NaN,left_only
4,chr1,1848381,.,G,A,2.0,0.045,44.0,rs914213962,0.000200,0.000200,0.000038,True,0.000134,NaN,left_only


In [50]:
df_common_merged[df_common_merged['_merge'] == 'both'].head()

,CHROM,POS,ID,REF,ALT,AC,AF,AN,avsnp150,gnomAD_genome_ALL,gnomAD_genome_NFE,pval,B-H_reject_H0,corrected_p_value,Num variant,_merge
11,chr1,6101717,.,A,C,4.0,0.087,46.0,rs867000634,0.0053,0.0058,1.520208e-04,True,4.569652e-04,12.0,both
12,chr1,6101720,.,A,C,4.0,0.087,46.0,.,0.0012,0.0003,1.308543e-09,True,9.462634e-09,10.0,both
13,chr1,6101722,.,T,C,4.0,0.091,44.0,.,0.0010,0.0000,0.000000e+00,True,0.000000e+00,11.0,both
14,chr1,6101725,.,T,C,3.0,0.068,44.0,.,0.0007,0.0003,3.543050e-07,True,2.024257e-06,11.0,both
69,chr1,37082151,.,G,A,4.0,0.091,44.0,.,0.0008,0.0011,1.918805e-07,True,1.119008e-06,5.0,both


In [51]:
len(df_common_merged[df_common_merged['_merge'] == 'both'])

166

In [52]:
ad_only = df_common_merged[df_common_merged['_merge'] == 'left_only']

In [56]:
len(ad_only[ad_only['B-H_reject_H0']])

1413

In [57]:
ad_only[['CHROM', 'POS', 'ID', 'REF', 'ALT', 'AC', 'AF', 'AN', 
         'avsnp150', 'gnomAD_genome_ALL', 'gnomAD_genome_NFE', 
         'pval', 'B-H_reject_H0', 'corrected_p_value']].to_csv('out_files/Lib1-6.no113.SNPs.hard_filtered.selected.normalized.hg38_multianno.nomissing.gnomad_below_0.01.ac_above_1.FDR0.01.below_0.2_glioma_samples.csv', 
                                                               sep='\t', index=False)